In [126]:
import pandas as pd

In [127]:
tp_cols = ["siape", "formacao", "tipo_jornada_trabalho", "vinculo", "lotacao", "admissao"]

tp_df = pd.read_csv("./perfis/docentes.csv", sep=";")
tp_df = tp_df[tp_cols]

tp_df.head()

,siape,formacao,tipo_jornada_trabalho,vinculo,lotacao,admissao
0,1543339,MESTRADO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA,2006/07/24 00:00:00.000000000
1,1554468,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,2008/09/12 00:00:00.000000000
2,1177821,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA DE MÚSICA,1998/04/28 00:00:00.000000000
3,2360824,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,2017/01/25 00:00:00.000000000
4,2364334,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,2009/10/13 00:00:00.000000000


In [128]:
tp_df.tail()

,siape,formacao,tipo_jornada_trabalho,vinculo,lotacao,admissao
2765,4246363,DOUTORADO,Dedicação exclusiva,Professor Visitante,INSTITUTO DE POLÍTICAS PÚBLICAS,2023/05/23 00:00:00.000000000
2766,3304576,DOUTORADO,Dedicação exclusiva,Professor Visitante,ESCOLA DE CIÊNCIAS E TECNOLOGIA,2022/08/12 00:00:00.000000000
2767,1056188,DOUTORADO,Dedicação exclusiva,Professor Visitante,DEPARTAMENTO DE BIOLOGIA CELULAR E GENÉTICA,2022/10/03 00:00:00.000000000
2768,3330361,DOUTORADO,Dedicação exclusiva,Professor Visitante,INSTITUTO METROPOLE DIGITAL,2023/02/15 00:00:00.000000000
2769,3309092,DOUTORADO,Dedicação exclusiva,Professor Visitante,DEPARTAMENTO DE GEOFÍSICA,2022/09/06 00:00:00.000000000


In [129]:
tp_df.dtypes

siape                     int64
formacao                 object
tipo_jornada_trabalho    object
vinculo                  object
lotacao                  object
admissao                 object
dtype: object

In [130]:
tp_df.describe()

,siape
count,2.770000e+03
mean,2.114588e+06
std,1.142222e+06
min,1.274600e+04
25%,1.297595e+06
50%,1.810985e+06
75%,2.722937e+06
max,9.350807e+06


In [131]:
qualis = ["siape","revista_a1","revista_a2","revista_b1","revista_b2","revista_b3","revista_b4","revista_b5","revista_c"]

ti_df_list = []
for year in range(2010,2021):
    ti_df_y = pd.read_csv("./indicadores/indicadores-pesquisa-" + str(year) + ".csv", sep=";")
    ti_df_y = ti_df_y[qualis]
    ti_df_list.append(ti_df_y)

ti_df = pd.concat(ti_df_list)
ti_df = ti_df.groupby("siape", as_index=False).sum()
ti_df.describe()


,siape,revista_a1,revista_a2,revista_b1,revista_b2,revista_b3,revista_b4,revista_b5,revista_c
count,2.756000e+03,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000
mean,2.112227e+06,2.634978,1.933599,1.750000,1.269231,0.818215,0.568578,0.054790,0.889332
std,1.137681e+06,6.073799,3.917333,5.375914,2.891393,2.413658,1.778480,0.330389,3.855806
min,1.274600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.296285e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.808676e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.721404e+06,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,9.350807e+06,71.000000,51.000000,99.000000,46.000000,41.000000,32.000000,8.000000,124.000000


In [132]:
def num_semestres(data:str, data_atual:str):
    data = data[:10]
    anos = int(data_atual[6:]) - int(data[:4])
    if int(data_atual[3:5]) < 7 and int(data[5:7]) < 7 or int(data_atual[3:5]) >= 7 and int(data[5:7]) >= 7:
        return 2*anos
    elif int(data_atual[3:5]) < 7 and int(data[5:7]) > 7:
        return 2*anos - 1
    else:
        return 2*anos + 1

df = tp_df.merge(ti_df, on="siape", how="inner")
del df["siape"]

data_atual = "18/10/2023"

df['num_semestres'] = df['admissao'].apply(lambda x: num_semestres(x, data_atual))
df = df[["formacao", "tipo_jornada_trabalho", "vinculo", "lotacao", "num_semestres", "revista_a1", "revista_a2", "revista_b1", "revista_b2", "revista_b3", "revista_b4", "revista_c"]]

df.head()

,formacao,tipo_jornada_trabalho,vinculo,lotacao,num_semestres,revista_a1,revista_a2,revista_b1,revista_b2,revista_b3,revista_b4,revista_c
0,MESTRADO,Dedicação exclusiva,Ativo Permanente,NÚCLEO DE EDUCAÇÃO DA INFÂNCIA,34,0,0,0,0,0,0,0
1,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,30,1,1,5,0,0,4,0
2,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA DE MÚSICA,51,0,0,0,0,0,0,0
3,MESTRADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,13,0,0,0,0,0,0,0
4,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ,28,0,0,0,1,0,0,0


In [133]:
df.dtypes

formacao                 object
tipo_jornada_trabalho    object
vinculo                  object
lotacao                  object
num_semestres             int64
revista_a1                int64
revista_a2                int64
revista_b1                int64
revista_b2                int64
revista_b3                int64
revista_b4                int64
revista_c                 int64
dtype: object

In [134]:
print("existem", len(df), "entradas diferentes;")
print("existem", len(df["formacao"].unique()), "formacoes diferentes;")
print("existem", len(df["tipo_jornada_trabalho"].unique()), "tipos de jornada de trabalho diferentes;")
print("existem", len(df["vinculo"].unique()), "tipos de vinculo diferentes;")
print("existem", len(df["lotacao"].unique()), "lotacoes diferentes;")
print("existem", len(df["num_semestres"].unique()), "datas de admissao diferentes;")

existem 2753 entradas diferentes;
existem 6 formacoes diferentes;
existem 3 tipos de jornada de trabalho diferentes;
existem 8 tipos de vinculo diferentes;
existem 135 lotacoes diferentes;
existem 89 datas de admissao diferentes;


In [135]:
for formacao in df["formacao"].unique():
    print(formacao, ": ", len(df[df["formacao"] == formacao]), sep="")

MESTRADO: 431
DOUTORADO: 2189
ESPECIALIZAÇÃO: 120
GRADUAÇÃO: 11
PÓS-DOUTORADO: 1
DESCONHECIDA: 1


In [136]:
df = df[df["formacao"] != "DESCONHECIDA"]
for formacao in df["formacao"].unique():
    print(formacao, ": ", len(df[df["formacao"] == formacao]), sep="")

MESTRADO: 431
DOUTORADO: 2189
ESPECIALIZAÇÃO: 120
GRADUAÇÃO: 11
PÓS-DOUTORADO: 1


In [137]:
for tipo_jornada_trabalho in df["tipo_jornada_trabalho"].unique():
    print(tipo_jornada_trabalho, ": ", len(df[df["tipo_jornada_trabalho"] == tipo_jornada_trabalho]), sep="")

Dedicação exclusiva           : 2155
20 horas semanais             : 307
40 horas semanais             : 290


In [138]:
for vinculo in df["vinculo"].unique():
    print(vinculo, ": ", len(df[df["vinculo"] == vinculo]), sep="")

Ativo Permanente: 2376
Celetista: 1
Colaborador PCCTAE e Magistério Federal: 2
Excedente de lotação: 3
Exercicio provisorio: 18
Professor Substituto: 260
Professor Temporario: 50
Professor Visitante: 42


In [139]:
df = df[df["vinculo"] != "Celetista"]
df = df[df["vinculo"] != "Colaborador PCCTAE e Magistério Federal"]
df = df[df["vinculo"] != "Excedente de lotação"]
for vinculo in df["vinculo"].unique():
    print(vinculo, ": ", len(df[df["vinculo"] == vinculo]), sep="")

Ativo Permanente: 2376
Exercicio provisorio: 18
Professor Substituto: 260
Professor Temporario: 50
Professor Visitante: 42


In [141]:
df.describe()

,num_semestres,revista_a1,revista_a2,revista_b1,revista_b2,revista_b3,revista_b4,revista_c
count,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000,2746.000000
mean,25.367444,2.634377,1.932265,1.750546,1.268026,0.811726,0.567371,0.888201
std,20.846636,6.079585,3.920029,5.382835,2.893793,2.387709,1.775777,3.857700
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,30.000000,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000
max,97.000000,71.000000,51.000000,99.000000,46.000000,41.000000,32.000000,124.000000


In [142]:
df.describe(include="object")

,formacao,tipo_jornada_trabalho,vinculo,lotacao
count,2746,2746,2746,2746
unique,5,3,5,134
top,DOUTORADO,Dedicação exclusiva,Ativo Permanente,ESCOLA AGRÍCOLA DE JUNDIAÍ
freq,2183,2149,2376,120
